In [1]:
import os
import sys
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random
import requests
import xml.etree.ElementTree as ET
import csv

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

#import tensorflow_probability as tfp
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14770526856357074960
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6922698752
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9113417002716399420
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [3]:
sys.path.append('../codes')
from codes.load_utils import num_labels, time_change, setup_dir, find_xml_filenames, find_csv_filenames

ModuleNotFoundError: No module named 'codes'

In [ ]:
omizu_path = r"G:\JaimeMorales\Codes\omizunagidori\export\Omizunagidori"
umineko_path = r"G:\JaimeMorales\Codes\omizunagidori\export\Umineko"
database_path = r"G:\JaimeMorales\Codes\omizunagidori\database"
database_l_path = r"G:\JaimeMorales\Codes\omizunagidori\database\labels"
database_o_acc_path = r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori"
database_u_acc_path = r"G:\JaimeMorales\Codes\omizunagidori\database\umineko"

In [ ]:
def make_labels(paths, label_wr_dir = r'G:\JaimeMorales\Codes\biodata\database\labels', fn_end = 17):
    
    for path in paths:
        label_path = path
        label_dir_p, label_fn = os.path.split(label_path)
        wr_fn = label_fn[:fn_end]
        tree = ET.parse(label_path)
        root = tree.getroot()
        filename = os.path.join(label_wr_dir,wr_fn+'_labels.csv')
        
        with open(filename,"w") as f:            
            csv_writer = csv.writer(f)
            header = ["event_type","start", "end"]
            csv_writer.writerow(header)
            for labellist in root.iter("labellist"):
                timestampStart = labellist[1].text
                timestampStart = timestampStart.replace('-','')
                timestampStart = timestampStart.replace('T',' ')      
                timestampStart = timestampStart.replace('Z','')
                timestampEnd = labellist[2].text
                timestampEnd = timestampEnd.replace('-','')
                timestampEnd = timestampEnd.replace('T',' ')
                timestampEnd = timestampEnd.replace('Z','')

                row = [labellist[0].text, labellist[1].text, labellist[2].text]
                row = [labellist[0].text,timestampStart,timestampEnd]
                csv_writer.writerow(row)
            
        print('created labels for >>> ',filename)
        
    return

# Create label files

In [ ]:
label_path_o = os.path.join(omizu_path,'labels')
label_paths_o = find_xml_filenames(label_path_o)
make_labels(label_paths_o, os.path.join(database_l_path,'omizunagidori'), fn_end = 17)

In [ ]:
label_path_u = os.path.join(umineko_path,'labels')
label_paths_u = find_xml_filenames(label_path_u)
make_labels(label_paths_u, os.path.join(database_l_path,'umineko'), fn_end = 11)

# Separate sensors

## Omizunagidori filenames

### 2018 file names

In [ ]:
O_Y18_save_folder = os.path.join(database_o_acc_path,'2018')
O_Y18_raw_path = os.path.join(omizu_path,'raw')
O_Y18_raw_paths = find_csv_filenames(O_Y18_raw_path, suffix = ".csv", year = '2018')

### 2019 file names

In [ ]:
O_Y19_save_folder = os.path.join(database_o_acc_path,'2019')
O_Y19_raw_path = os.path.join(omizu_path,'raw')
O_Y19_raw_paths = find_csv_filenames(O_Y19_raw_path, suffix = ".csv", year = '2019')

### 2020 file names

In [ ]:
O_Y20_save_folder = os.path.join(database_o_acc_path,'2020')
O_Y20_raw_path = os.path.join(omizu_path,'raw')
O_Y20_raw_paths = find_csv_filenames(O_Y20_raw_path, suffix = ".csv", year = '2020')

### 2021 file names

In [ ]:
O_Y21_save_folder = os.path.join(database_o_acc_path,'2021')
O_Y21_raw_path = os.path.join(omizu_path,'raw')
O_Y21_raw_paths = find_csv_filenames(O_Y21_raw_path, suffix = ".csv", year = '2021')

### 2022 file names

In [ ]:
O_Y22_save_folder = os.path.join(database_o_acc_path,'2022')
O_Y22_raw_path = os.path.join(omizu_path,'raw')
O_Y22_raw_paths = find_csv_filenames(O_Y22_raw_path, suffix = ".csv", year = '2022')

### ALL Omizunagidori files

In [ ]:
O_all_save_folder = [O_Y18_save_folder, O_Y19_save_folder, O_Y20_save_folder, O_Y21_save_folder, O_Y22_save_folder]
O_all_raw_paths = [O_Y18_raw_paths, O_Y19_raw_paths, O_Y20_raw_paths, O_Y21_raw_paths, O_Y22_raw_paths]

## Umineko filenames

### 2018 file names

In [ ]:
U_Y18_save_folder = os.path.join(database_u_acc_path,'2018')
U_Y18_raw_path = os.path.join(umineko_path,'raw')
U_Y18_raw_paths = find_csv_filenames(U_Y18_raw_path, suffix = ".csv", year = '2018')

### 2019 file names

In [ ]:
U_Y19_save_folder = os.path.join(database_u_acc_path,'2019')
U_Y19_raw_path = os.path.join(umineko_path,'raw')
U_Y19_raw_paths = find_csv_filenames(U_Y19_raw_path, suffix = ".csv", year = '2019')

### 2022 file names

In [ ]:
U_Y22_save_folder = os.path.join(database_u_acc_path,'2022')
U_Y22_raw_path = os.path.join(umineko_path,'raw')
U_Y22_raw_paths = find_csv_filenames(U_Y22_raw_path, suffix = ".csv", year = '2022')

### All umineko files

In [ ]:
U_all_save_folder = [U_Y18_save_folder, U_Y19_save_folder, U_Y22_save_folder]
U_all_raw_paths = [U_Y18_raw_paths, U_Y19_raw_paths, U_Y22_raw_paths]

# Sensor specific dataframe creation

In [ ]:
def separate_by_sensor(filename, save_folder, sensor='acc', time_format="%Y%m%d_%H:%M:%S.%f"):
    data = pd.read_csv(filename, parse_dates=["timestamp"])
    data["timestamp"] = pd.to_datetime(data["timestamp"],format=time_format)
    
    #data['label'] = np.where(data['timestamp']=='NaN', 'unknown', data['label'])
    data.fillna('unlabeled',inplace=True)
    
    if sensor == 'acc':
        new_df = data.drop(['logger_id', 'latitude', 'longitude', 'gps_status', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x', 'mag_y', 'mag_z', 'illumination', 'pressure', 'temperature','activity_class'],axis=1)
    else:
        new_df = data
        
    name = filename[-11:-4]
    name = name.replace('_','')
    name = name.replace('00','')
    save_name = os.path.join(save_folder,'data_w_timestamps',name+'_acc.csv')
    new_df.to_csv(save_name,index=False)
    print('Saved to: ', save_name) 
    
    #data_df = pd.read_csv(save_name, parse_dates=["timestamp"])
    data_df = new_df
    #l = []
    #for i in range(len(data)):
    #    l.append(data_df['timestamp'][i].replace('+00:00',''))
    #data_df['timestamp'] = l
    #data_df['timestamp'][0] = data_df['timestamp'][0]+'.000000'
    
    #data_df = pd.to_datetime(data_df["timestamp"],format=time_format)
    
    data_df = time_change(data_df)
    
    #data_df.to_csv(os.path.join(save_folder,name+'t_acc.csv'),index=False)
    return data_df
    
def join_by_year(read_dir):

    all_bird_lst = []
    for file in os.listdir(os.path.join(read_dir,'data_w_timestamps')):
        if file.endswith(".csv"):
            path = os.path.join(read_dir, 'data_w_timestamps', file)
            print(path)
            df = pd.read_csv(path,parse_dates=["timestamp"])
            all_bird_lst.append(df)
            #all_bird_df = all_bird_df._append(df, ignore_index=True)
            #all_bird_df = pd.concat([all_bird_df,df])

    all_bird_df = pd.concat(all_bird_lst)

    birds = list(all_bird_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    print(birds)
    
    all_bird_df.to_csv(os.path.join(read_dir, 'all_bird_df.csv'),index=False)

    #print(all_bird_df)

    #all_bird_df.drop(['activity_class'],axis = 1,inplace = True)
    #all_bird_df['label'] = np.where(all_bird_df['timestamp']=='NaN', 'unknown', all_bird_df['label'])
    #all_bird_df.fillna('unknown',inplace=True)
    all_bird_df.dropna(inplace = True)
    all_bird_df.reset_index(inplace = True)
    all_bird_df.drop(['index'],axis = 1,inplace = True)
    all_bird_df.to_csv(os.path.join(read_dir, 'all_bird_df_Y' + os.path.split(read_dir)[-1] +'_WL.csv'),index=False)
    
    labels_df = all_bird_df.drop_duplicates(subset=['label'],keep = 'first')['label']
    labels_l = list(labels_df)
    print('labels' + os.path.split(read_dir)[-1]+':')
    print(labels_l)
    labels_df.to_csv(os.path.join(read_dir, 'label_df_Y' + os.path.split(read_dir)[-1] +'.csv'),index=False)
    
    return all_bird_df

## Create acc files - OMIZUNAGIDORI

In [ ]:
def create_acc_files(all_raw_paths,all_save_folder,bird_n):

    all_bird_Y_df_l = []
    year_df_t_l = []

    for i in range(len(all_save_folder)):
        bird_df_t_l =[]
        for raw_path in all_raw_paths[i]:
            print('Acc sensor df from: ', raw_path)
            bird_acc_df = separate_by_sensor(raw_path,all_save_folder[i])
            bird_df_t_l.append(bird_acc_df)
        year_df_t = pd.concat(bird_df_t_l)
        year_df_t_l.append(year_df_t)
        all_bird = join_by_year(all_save_folder[i])
        print('Joined all sensors for: ', all_save_folder[i])
        all_bird_Y_df_l.append(all_bird)

### Join all birds by year with Unix time

    all_t_df = pd.concat(year_df_t_l)
    #all_t_df.drop(['activity_class'], axis=1, inplace=True)
    all_t_df.dropna(inplace=True)
    all_t_df.reset_index(inplace = True)
    all_t_df.drop(['index'],axis = 1,inplace = True)
    #print('timechange dataframe: ')
    #print(all_t_df)

    all_df = pd.concat(all_bird_Y_df_l)
    labels_df = pd.DataFrame(all_df.drop_duplicates(subset=['label'],keep = 'first')['label'])
    labels_df.reset_index(inplace=True)
    labels_df.drop(['index'],axis=1,inplace=True)

    if bird_n=='omizunagidori':
        all_t_df.to_csv(os.path.join(database_o_acc_path,'Omizu_all_t_df.csv'), index = False)
        labels_df.to_csv(os.path.join(database_l_path,'O_labels_df.csv'),index = False)
    elif bird_n=='umineko':
        all_t_df.to_csv(os.path.join(database_u_acc_path,'Umineko_all_t_df.csv'), index = False)
        labels_df.to_csv(os.path.join(database_l_path,'U_labels_df.csv'),index = False)

### Join all birds by year with timestamp


In [ ]:
O_all_bird_Y_df_l = []
year_df_t_l = []

for i in range(len(O_all_save_folder)):
    bird_df_t_l =[]
    for raw_path in O_all_raw_paths[i]:
        print('Acc sensor df from: ', raw_path)
        bird_acc_df = separate_by_sensor(raw_path,O_all_save_folder[i])
        bird_df_t_l.append(bird_acc_df)
    year_df_t = pd.concat(bird_df_t_l)
    year_df_t_l.append(year_df_t)
    all_bird = join_by_year(O_all_save_folder[i])
    print('Joined all sensors for: ', O_all_save_folder[i])
    O_all_bird_Y_df_l.append(all_bird)

### Join all birds by year with Unix time

Omizu_all_t_df = pd.concat(year_df_t_l)
#Omizu_all_t_df.drop(['activity_class'], axis=1, inplace=True)
Omizu_all_t_df.dropna(inplace=True)
Omizu_all_t_df.reset_index(inplace = True)
Omizu_all_t_df.drop(['index'],axis = 1,inplace = True)
#print('timechange dataframe: ')
#print(Omizu_all_t_df)
Omizu_all_t_df.to_csv(os.path.join(database_o_acc_path,'Omizu_all_t_df.csv'), index = False)

### Join all birds by year with timestamp

Omizu_all_df = pd.concat(O_all_bird_Y_df_l)
O_labels_df = pd.DataFrame(Omizu_all_df.drop_duplicates(subset=['label'],keep = 'first')['label'])
O_labels_df.reset_index(inplace=True)
O_labels_df.drop(['index'],axis=1,inplace=True)
O_labels_df.to_csv(os.path.join(database_l_path,'O_labels_df.csv'),index = False)

## Create acc files - UMINEKO

In [ ]:
U_all_bird_Y_df_l = []
year_df_t_l = []

for i in range(len(U_all_save_folder)):
    bird_df_t_l =[]
    for raw_path in U_all_raw_paths[i]:
        print('Acc sensor df from: ', raw_path)
        print('Saved to: ', U_all_save_folder[i]) 
        bird_acc_df = separate_by_sensor(raw_path,U_all_save_folder[i])
        bird_df_t_l.append(bird_acc_df)
    year_df_t = pd.concat(bird_df_t_l)
    year_df_t_l.append(year_df_t)
    all_bird = join_by_year(U_all_save_folder[i])
    U_all_bird_Y_df_l.append(all_bird)

### Join all birds by year with Unix time

Umineko_all_t_df = pd.concat(year_df_t_l)
#Umineko_all_t_df.drop(['activity_class'], axis=1, inplace=True)
Umineko_all_t_df.dropna(inplace=True)
Umineko_all_t_df.reset_index(inplace = True)
Umineko_all_t_df.drop(['index'],axis = 1,inplace = True)
#print('timechange dataframe: ')
#print(Umineko_all_t_df)
Umineko_all_t_df.to_csv(os.path.join(database_u_acc_path,'Umineko_all_t_df.csv'), index = False)

### Join all birds by year with timestamp

umineko_all_df = pd.concat(U_all_bird_Y_df_l)
U_labels_df = pd.DataFrame(umineko_all_df.drop_duplicates(subset=['label'],keep = 'first')['label'])
U_labels_df.reset_index(inplace=True)
U_labels_df.drop(['index'],axis=1,inplace=True)
U_labels_df.to_csv(os.path.join(database_l_path,'U_labels_df.csv'),index = False)

# Assign labels, separate years, separate birds

## Year times

In [ ]:
def divide_years(acc_df,write_folder,year_times):
    
    new_df = acc_df
    new_df['year']='0'
    years = list(year_times.keys())

    for year in years:
        print(year_times[year][0])
        year_df = new_df[new_df['timestamp']>year_times[year][0]]
        year_df = year_df[year_df['timestamp']<year_times[year][1]]
        new_df['year'] = np.where(new_df['timestamp'].between(year_times[year][0],year_times[year][1]), year, new_df['year'])
        year_df = new_df[new_df['year']==year]
        year_df.to_csv(os.path.join(write_folder,year,'l_'+year+'_acc.csv'),index = False)

In [ ]:
def divide_birds(read_folder,year_times):
    
    years = list(year_times.keys())

    for year in years:
        year_df = pd.read_csv(os.path.join(read_folder,year,'l_'+year+'_acc.csv'))
        birds_df = pd.DataFrame(year_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
        birds = list(birds_df['animal_tag'])
        print('year: ',year)
        for bird in birds:
            print('bird: '+bird)
            new_df = year_df[year_df['animal_tag']==bird]
            new_df.to_csv(os.path.join(read_folder,year,'labeled',bird+'_l_acc.csv'),index=False)

### Load labels

In [ ]:
omizu_labels_all_df = pd.read_csv(os.path.join(database_l_path,'O_labels_df.csv'))
omizu_l_dict = omizu_labels_all_df['label'].to_dict()

In [ ]:
umineko_labels_all_df = pd.read_csv(os.path.join(database_l_path,'U_labels_df.csv'))
umineko_l_dict = umineko_labels_all_df['label'].to_dict()

## Omizunagidori

In [ ]:
o_year_times = {'2018':[1514768460000,1546304460000],'2019':[1546304460000,1577840460000],'2020':[1577840460000,1609462860000],'2021':[1609462860000,1640998860000],'2022':[1640998860000,1672534860000]}

In [ ]:
Omizu_all_t_df = pd.read_csv(os.path.join(database_o_acc_path,'omizu_all_t_df.csv'))
omizu_lab_all_df = num_labels(Omizu_all_t_df,omizu_labels_all_df)
divide_years(omizu_lab_all_df,database_o_acc_path,o_year_times)
divide_birds(database_o_acc_path,o_year_times)

## Umineko

In [ ]:
u_year_times = {'2018':[1514768460000,1546304460000],'2019':[1546304460000,1577840460000],'2022':[1640998860000,1672534860000]}

In [ ]:
Umineko_all_t_df = pd.read_csv(os.path.join(database_u_acc_path,'umineko_all_t_df.csv'))
umineko_lab_all_df = num_labels(Umineko_all_t_df,umineko_labels_all_df)
divide_years(umineko_lab_all_df,database_u_acc_path,u_year_times)
divide_birds(database_u_acc_path,u_year_times)

# verifylabels

In [ ]:
year = '2022'
o_labels_2022 = pd.read_csv(os.path.join(database_l_path,'omizunagidori','Omizunagidori'+year+'_labels.csv'),parse_dates=['start','end'])
t_labels = time_change(o_labels_2022,column='start')
changed_t_labels = time_change(t_labels,column='end')
changed_t_labels

In [ ]:
print(os.path.join(database_o_acc_path,year,'l_'+year+'_acc.csv'))
data_l_df = pd.read_csv(os.path.join(database_o_acc_path,year,'l_'+year+'_acc.csv'))
data_l_df

# Vizualize transformed data
## Load DataFrame

In [ ]:
viz_path = r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv"

In [ ]:
viz_df = pd.read_csv(viz_path,parse_dates=["timestamp"])
viz_df

In [ ]:
bird_df = viz_df[viz_df['animal_tag']=='LB01']
bird_df

In [ ]:
bird_l_df = bird_df[bird_df['label']=='unknown']
bird_l_df

In [ ]:
view_df = bird_l_df

In [ ]:
def plot_two_data_segments(data, start_idx, lenght, label_name = "l_val"):
    
    fig = plt.figure(num=1, figsize=(50,30), dpi=300)
    axL = fig.add_axes([0, 0, 1, 1])
    plot_one_data_segment(axL, data, start_idx, lenght, label_name = label_name)
    fig.savefig(r"G:\JaimeMorales\Codes\omizunagidori\figures\timeseries.svg", format = 'svg', dpi=500, bbox_inches = 'tight')
    fig.tight_layout()
    fig.show()
    
def plot_one_data_segment(ax, data, start_idx, length, label_name = "label"):
    if start_idx + length > len(data) - 1:
        start_idx = len(data) - 1 - length
    if start_idx < 0:
        start_idx = 0
    ax.plot(data["timestamp"][start_idx:start_idx + length], data['acc_x'][start_idx:start_idx + length], '-' , color = 'red', linewidth=12)
    ax.plot(data["timestamp"][start_idx:start_idx + length], data['acc_y'][start_idx:start_idx + length], '-' , color = 'green', linewidth=12)
    ax.plot(data["timestamp"][start_idx:start_idx + length], data['acc_z'][start_idx:start_idx + length], '-' , color = 'blue', linewidth=12)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', bottom=False, top=False, labelbottom=False)
    #ax.set_xlabel("time [s]")

## Plot dataset

In [ ]:
plot_two_data_segments(viz_df, 12000, 2000)